In [1]:
# Core
import numpy as np
import pandas as pd

# Astronomy
import rebound

In [2]:
# Local imports
import kepler_sieve
from horizons import make_sim_horizons
from rebound_sim import make_sim_planets
from rebound_integrate import integrate_numpy, integrate_df
from db_config import db_engine
from db_utils import sp2df, get_columns, df2db, df2csv, csv2db, csvs2db

In [3]:
from asteroid_element import get_ast_ref_elts, get_ast_ref_elts_jpl, add_asteroid_elts, update_asteroid_elements
from asteroid_element_synch import process_dates, calc_ref_elt

In [4]:
# Typing
from typing import Tuple, List

In [6]:
epoch_out = 59000
elt_dates = sp2df(sp_name='JPL.GetAsteroidRefElementDates', params={'epoch':epoch_out})

In [7]:
elt_dates

,TimeID,epoch,AsteroidCount
0,63678240,44221.0,1
1,64319040,44666.0,1
2,64334880,44677.0,1
3,68271840,47411.0,1
4,68562720,47613.0,1
...,...,...,...
5229,84951360,58994.0,5
5230,84952800,58995.0,4
5231,84955680,58997.0,3
5232,84957120,58998.0,2


In [9]:
mask = (elt_dates.epoch == 58600)
elt_dates[mask]

,TimeID,epoch,AsteroidCount
4909,84384000,58600.0,2


In [7]:
# First epoch
# epoch0 = int(elt_dates.epoch[0])
epoch0 = 58600

In [8]:
# Start with a base simulation of the Sun, Planets and Moon
sim = make_sim_planets(epoch=epoch0)
# Set simulation time t to match epoch
sim.t = sim.epoch
# Set the number of active particles to the base simulation
# https://rebound.readthedocs.io/en/latest/ipython/Testparticles.html
sim.N_active = sim.N

In [9]:
# Get the elements
# elts = sp2df(sp_name='KS.GetAsteroidRefElements', params={'epoch':epoch0})
elts = get_ast_ref_elts_jpl(epoch0)

In [10]:
elts = elts[0:2]
elts_orig = elts.copy()
elts

,AsteroidID,TimeID,AsteroidName,epoch,a,e,inc,Omega,omega,M,BodyID,BodyName
0,1,84384000,Ceres,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,1000001,SB.Ceres
1,4,84384000,Vesta,58600.0,2.361418,0.088721,0.124647,1.811840,2.630709,1.673106,1000004,SB.Vesta


In [69]:
sim.N

11

In [71]:
i=0
a = elts.a[i]
e = elts.e[i]
inc = elts.inc[i]
Omega = elts.Omega[i]
omega = elts.omega[i]
M = elts.M[i]

In [87]:
sim.particles[12].calculate_orbit(primary=primary).a

2.769165100000001

In [88]:
orbits = sim.calculate_orbits(primary=primary)

In [93]:
orbits[11].a

2.769165100000001

In [74]:
primary = sim.particles['Sun']
primary

<rebound.Particle object, m=1.0 x=-0.001699702116697901 y=0.007591941485824569 z=-3.294738263228371e-05 vx=-8.347175817777104e-06 vy=7.503821640682956e-07 vz=2.164051447319061e-07>

In [77]:
sim.add(m=0.0, a=a, e=e, inc=inc, Omega=Omega, omega=omega, M=M, primary=primary)
# sim.particles[-1].name = elts.AsteroidName[i]

In [84]:
sim.calculate_orbits(primary=primary)

<rebound.Particle object, m=0.0 x=-1.358266863358887 y=-2.3650681547556127 z=0.17534354611513353 vx=0.00845671670723955 vy=-0.005875569796473027 vz=-0.0017455417801408722>

In [85]:
sim.particles[12].a

2.7498584752273363

In [11]:
# First set of asteroid IDs
asteroid_ids = add_asteroid_elts(sim=sim, elts=elts)

In [12]:
# sim.t
sim.particles[11].a

2.7498584752273363

In [34]:
asteroid_ids

array([1, 4])

In [13]:
elt_dates2 = elt_dates.loc[elt_dates.epoch==58600]
elt_dates2

,TimeID,epoch,AsteroidCount
4912,84384000,58600.0,2


In [39]:
# Integrate to the next epoch
# sim.integrate(epoch_next, exact_finish_time=1)
# Update the asteroid elements as of this epoch
update_asteroid_elements(sim=sim, elts=elts, epoch=epoch0)


In [14]:
elts

,AsteroidID,TimeID,AsteroidName,epoch,a,e,inc,Omega,omega,M,BodyID,BodyName
0,1,84384000,Ceres,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,1000001,SB.Ceres
1,4,84384000,Vesta,58600.0,2.361418,0.088721,0.124647,1.811840,2.630709,1.673106,1000004,SB.Vesta


In [15]:
elts.a - elts_orig.a

0    0.0
1    0.0
Name: a, dtype: float64

In [17]:
sim, elts = process_dates(epoch_out, elt_dates=elt_dates2, max_dates=1)

In [24]:
elts

,AsteroidID,TimeID,AsteroidName,epoch,a,e,inc,Omega,omega,M,BodyID,BodyName
0,1,84384000,Ceres,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,1000001,SB.Ceres
1,4,84384000,Vesta,58600.0,2.361418,0.088721,0.124647,1.811840,2.630709,1.673106,1000004,SB.Vesta


In [32]:
max_date = 3
N_date = 1000

In [12]:
ref_elts = calc_ref_elt(sim)

In [13]:
ref_elts

,AsteroidID,TimeID,epoch,a,e,inc,Omega_node,omega_peri,f,M,d,v,h,period,mean_motion,T_peri,pomega
0,1413593.0,84960000,59000.0,2.319947,0.193103,0.013013,3.085343,3.471796,5.640864,5.847385,1.934352,0.013366,0.025725,1289.805987,0.004871,59089.460562,0.273954
1,1413594.0,84960000,59000.0,2.243868,0.265620,0.169600,3.559730,2.611393,3.884723,4.302661,2.592652,0.009825,0.024859,1226.883202,0.005121,59386.726134,6.171122
2,1000007.0,84960000,59000.0,3.185015,0.298391,0.164023,2.169598,2.773325,1.921288,1.322903,3.232631,0.009502,0.029321,2074.791898,0.003028,58563.159785,4.942923


In [17]:
schema='KS'
table='AsteroidElement_Ref'
truncate=False
chunksize=0
verbose=True
df2db(df=ref_elts, schema=schema, table=table, truncate=truncate, chunksize=chunksize, verbose=verbose)

Extracting 3 records from DataFrame into CSV files in chunks of 0 rows...
CSV file name: ../data/df2db/AsteroidElement_Ref/AsteroidElement_Ref.csv
Elapsed Time for CSV Conversion: 0:00:00.003431
Elapsed Time for DB insertion: 0:00:00.089094
